In [4]:

cd ../../../../

/Users/mykhailoslukvin/repo/dv-data-pipeline


/Users/mykhailoslukvin/.local/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [5]:
pwd

'/Users/mykhailoslukvin/repo/dv-data-pipeline'

In [6]:
import requests
import aiohttp
import asyncio
import requests
import pandas as pd
import io
import os
import json
from collections import defaultdict
import country_converter as coco
import pandas as pd

from dfpp.storage import StorageManager
from dfpp.storage.postgres import AsyncPGClient

from dfpp.transformation.series_disambiguation import filter_important_dimensions
from dfpp.transformation.column_name_template import SexEnum

cc = coco.CountryConverter()

MAX_CONCURRENCY = 5

In [7]:
def get_indicators():
    url = "https://unstats.un.org/sdgapi/v1/sdg/CompareTrends/GetDisaggregatedGlobalAndRegional"

    headers = {"Accept": "application/json"}

    response = requests.post(url, headers=headers)
    response.raise_for_status()

    data = response.json()
    return data

In [8]:
indicators = get_indicators()

In [9]:
series_codes = set([i["seriesCode"] for i in indicators])

In [14]:
async def fetch_data(session, series_id, semaphore):
    url = f"https://unstats.un.org/sdgapi/v1/sdg/Series/Data"
    params = {"seriesCode": series_id, "page": 1, "pageSize": 10000000}
    headers = {"Accept": "application/json"}

    async with semaphore:
        async with session.get(url, headers=headers, params=params) as response:
            response.raise_for_status()
            return await response.json()


async def process_series(series_id, session, semaphore):
    year_data = await fetch_data(session, series_id, semaphore)

    dimensions = [d["id"] for d in year_data["dimensions"]]
    size = year_data["totalElements"]
    return series_id, year_data["data"], dimensions, size


async def get_series_data_and_dimensions(
    series_codes, max_concurrent_requests=MAX_CONCURRENCY
):
    semaphore = asyncio.Semaphore(max_concurrent_requests)
    series_data_map = {}
    series_map = defaultdict(dict)

    async with aiohttp.ClientSession() as session:
        tasks = []
        for row in series_codes:
            series_id = row
            tasks.append(process_series(series_id, session, semaphore))

        results = await asyncio.gather(*tasks)

        for series_id, data, dimensions, size in results:
            series_data_map[series_id] = data
            series_map[series_id]["dimensions"] = dimensions
            series_map[series_id]["totalElements"] = size

    return series_data_map, series_map

In [456]:
sex_remap = {
    "BOTHSEX": SexEnum.BOTH.value,
    "MALE": SexEnum.MALE.value,
    "FEMALE": SexEnum.FEMALE.value,
}

age_remap = {"ALLAGE", "all"}

In [457]:
series_data_map, dimension_map = await get_series_data_and_dimensions(
    list(series_codes)[:20]
)

In [454]:
for series_id, data in series_data_map.items():
    df_source = pd.DataFrame(data)

    df = df_source.copy()

    df.shape
    assert df.shape[0] == dimension_map[series_id]["totalElements"]
    df.columns

    df_dimensions = pd.json_normalize(df["dimensions"])
    df_attributes = pd.json_normalize(df["attributes"])

    df = pd.concat([df.drop(columns=["dimensions"]), df_dimensions], axis=1)
    df = pd.concat([df.drop(columns=["attributes"]), df_attributes], axis=1)
    df.columns
    df.head()
    df.seriesDescription.iloc[0]
    assert df.series.value_counts(dropna=False).shape[0] == 1
    df.rename(
        columns={
            "geoAreaCode": "alpha_3_code",
            "geoAreaName": "country_or_area",
            "timePeriodStart": "year",
        },
        inplace=True,
    )

    original_dimension_columns = df_dimensions.columns
    dimension_columns = original_dimension_columns.str.lower().str.replace(
        "\s", "_", regex=True
    )
    dimension_column_rename_map = dict(
        zip(original_dimension_columns, dimension_columns)
    )
    dimension_column_rename_map
    dimension_columns
    df.rename(columns=dimension_column_rename_map, inplace=True)
    assert df.shape[0] > 0
    df_c = df.copy()
    df_selection, columns_to_keep = filter_important_dimensions(
        df_c,
        base_columns=["country_or_area", "alpha_3_code", "year"],
        important_columns=list(dimension_columns),
        value_column="value",
    )
    columns_to_keep
    
    if "age" in df_selection.columns:
        df_selection["age"] = df_selection["age"].replace()

    if "sex" in df_selection.columns:
        df_selection["sex"] = df_selection["sex"].replace(sex_remap)

    df_selection["alpha_3_code"] = cc.pandas_convert(
        series=df_selection["alpha_3_code"], src="ISOnumeric", to="ISO3"
    )
    df_selection.alpha_3_code.value_counts(dropna=False)
    df_selection.head(10)

2 not found in ISOnumeric
142 not found in ISOnumeric
53 not found in ISOnumeric
29 not found in ISOnumeric
13 not found in ISOnumeric
62 not found in ISOnumeric
143 not found in ISOnumeric
14 not found in ISOnumeric
753 not found in ISOnumeric
30 not found in ISOnumeric
151 not found in ISOnumeric
150 not found in ISOnumeric
513 not found in ISOnumeric
432 not found in ISOnumeric
419 not found in ISOnumeric
199 not found in ISOnumeric
54 not found in ISOnumeric
57 not found in ISOnumeric
17 not found in ISOnumeric
15 not found in ISOnumeric
747 not found in ISOnumeric
21 not found in ISOnumeric
154 not found in ISOnumeric
543 not found in ISOnumeric
61 not found in ISOnumeric
722 not found in ISOnumeric
5 not found in ISOnumeric
35 not found in ISOnumeric
18 not found in ISOnumeric
34 not found in ISOnumeric
39 not found in ISOnumeric
202 not found in ISOnumeric
11 not found in ISOnumeric
145 not found in ISOnumeric
155 not found in ISOnumeric
1 not found in ISOnumeric
53 not found in

In [455]:
df_selection

,country_or_area,alpha_3_code,year,value
0,Afghanistan,AFG,2016.0,35
1,Afghanistan,AFG,2017.0,35
2,Afghanistan,AFG,2018.0,35
3,Afghanistan,AFG,2019.0,35
4,Afghanistan,AFG,2020.0,40
...,...,...,...,...
1792,Zimbabwe,ZWE,2018.0,45
1793,Zimbabwe,ZWE,2019.0,45
1794,Zimbabwe,ZWE,2020.0,50
1795,Zimbabwe,ZWE,2021.0,60
